# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [1]:
import sympy as sp
from ham_to_sparse_matrix import *
from constants import *
import scipy.sparse.linalg

In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(2*aLat) )
fermionNI=sp.expand( (xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [5]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [6]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 - 0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [7]:
ham=0

lam=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    return lam*qs[n]*qs[n]*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
ham

(4*\chi^{\dagger}_{0}*\chi_{1} - 4*\chi^{\dagger}_{0}*\chi_{2} + 4*\chi^{\dagger}_{1}*\chi_{0} + 4*\chi^{\dagger}_{1}*\chi_{2} - 4*\chi^{\dagger}_{2}*\chi_{0} + 4*\chi^{\dagger}_{2}*\chi_{1} + 4*p_{0}**2 + 4*p_{1}**2 + 4*p_{2}**2 - q_{0}*q_{1} - q_{0}*q_{2} + 14.0*q_{0}**2 - 24*q_{0}**2*\chi^{\dagger}_{0}*\chi_{0} + 4*q_{0}**3*q_{1} - 4*q_{0}**3*q_{2} + 4*q_{0}**6 - q_{1}*q_{0} - q_{1}*q_{2} - 10.0*q_{1}**2 + 24*q_{1}**2*\chi^{\dagger}_{1}*\chi_{1} - 4*q_{1}**3*q_{0} + 4*q_{1}**3*q_{2} + 4*q_{1}**6 - q_{2}*q_{0} - q_{2}*q_{1} + 14.0*q_{2}**2 - 24*q_{2}**2*\chi^{\dagger}_{2}*\chi_{2} + 4*q_{2}**3*q_{0} - 4*q_{2}**3*q_{1} + 4*q_{2}**6)/8

In [8]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

\chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{0}*\chi_{2}/2 + \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{0}/2 + \chi^{\dagger}_{2}*\chi_{1}/2 - a^{\dagger}_{0}*a^{\dagger}_{1}/16 - a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 9*a^{\dagger}_{0}*a_{0}/8 - 3*a^{\dagger}_{0}*a_{0}*\chi^{\dagger}_{0}*\chi_{0}/2 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a^{\dagger}_{1}/8 - a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a^{\dagger}_{2}/8 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}**2*a^{\dagger}_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}**3/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{1}/8 - a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{2}/8 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2*a_{0}*a^{\dagger}_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2*a_{0}**2/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**3*a_{0}/16 + a^{\

In [21]:
tstTerm=aops[3]**6/16
mat=convert_term_to_matrix(tstTerm,7,N,aops,adags,xs,xdags)
mat

<2744x2744 sparse matrix of type '<class 'numpy.object_'>'
	with 392 stored elements in Compressed Sparse Row format>

In [22]:
print(mat)

  (0, 12)	1.67705094814301
  (1, 13)	1.67705094814301
  (14, 26)	1.67705094814301
  (15, 27)	1.67705094814301
  (28, 40)	1.67705094814301
  (29, 41)	1.67705094814301
  (42, 54)	1.67705094814301
  (43, 55)	1.67705094814301
  (56, 68)	1.67705094814301
  (57, 69)	1.67705094814301
  (70, 82)	1.67705094814301
  (71, 83)	1.67705094814301
  (84, 96)	1.67705094814301
  (85, 97)	1.67705094814301
  (98, 110)	1.67705094814301
  (99, 111)	1.67705094814301
  (112, 124)	1.67705094814301
  (113, 125)	1.67705094814301
  (126, 138)	1.67705094814301
  (127, 139)	1.67705094814301
  (140, 152)	1.67705094814301
  (141, 153)	1.67705094814301
  (154, 166)	1.67705094814301
  (155, 167)	1.67705094814301
  (168, 180)	1.67705094814301
  :	:
  (2563, 2575)	1.67705094814301
  (2576, 2588)	1.67705094814301
  (2577, 2589)	1.67705094814301
  (2590, 2602)	1.67705094814301
  (2591, 2603)	1.67705094814301
  (2604, 2616)	1.67705094814301
  (2605, 2617)	1.67705094814301
  (2618, 2630)	1.67705094814301
  (2619, 2631)	1.677

In [9]:
for cutoff in [2,3,4,5,6,7,8,9,10,11,12]:
    t=Timer('toHam')
    t.start()
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    t.stop()
    
    t=Timer('evals')
    t.start()
    ens=scipy.sparse.linalg.eigs(hamMat,k=10,sigma=0)[0]
    t.stop()
    
    print("Lambda = ",cutoff)
    print(ens)
    print()

toHam timer took: 3.1003 seconds
evals timer took: 0.0020 seconds
Lambda =  2
[ 0.0500414 +2.4659816e-10j  0.05004131+2.2609234e-09j
  0.05004135-4.4795953e-10j  0.05004134-2.7546445e-09j
 -0.4499581 +1.2257945e-08j  0.1874998 -6.1010230e-10j
  0.18749993-1.9037238e-09j -0.4499581 -7.2593931e-09j
  0.05004137+4.7447806e-09j  0.05004138-5.9703336e-09j]

toHam timer took: 4.4757 seconds
evals timer took: 0.0032 seconds
Lambda =  3
[0.04553828+7.3646550e-10j 0.05645382-9.4140096e-10j
 0.17575878-1.9280917e-09j 0.18965451+3.5438590e-09j
 0.18291551+9.1546948e-10j 0.23131132-6.7541781e-09j
 0.4234793 +1.8205257e-09j 0.41870093-1.5271187e-09j
 0.55000836-2.3337959e-08j 0.57022065+2.8228075e-10j]

toHam timer took: 7.4137 seconds
evals timer took: 0.0064 seconds
Lambda =  4
[0.16245367-9.6729620e-09j 0.6318225 -4.1816008e-09j
 0.8892409 -7.5551139e-09j 1.0975865 -9.0338563e-09j
 1.095807  -7.9006313e-09j 1.3383268 -1.5619836e-08j
 1.3511088 +1.4993645e-08j 1.5561867 -2.0725325e-08j
 1.5650226

KeyboardInterrupt: 